In [3]:
import os

import pandas as pd

import experiment
import fluffi

# Constants
RUN_DIR = os.path.join(experiment.EXP_BASE_DIR, "run1/")


In [4]:
exp = {}

# Iterate over locations
for n in range(experiment.N_MIN, experiment.N_MAX + 1):
    location = fluffi.LOCATION_FMT.format(n)
    exp[location] = {}
    location_dir = os.path.join(RUN_DIR, location)

    # Iterate over benchmarks
    for benchmark in experiment.BENCHMARKS:
        exp[location][benchmark] = []
        benchmark_dir = os.path.join(location_dir, benchmark)

        # Get the trials
        for filename in os.listdir(benchmark_dir):
            if not filename.endswith(".parquet"):
                continue
            parquet_path = os.path.join(benchmark_dir, filename)

            # Read from parquet
            df = pd.read_parquet(parquet_path)
            exp[location][benchmark].append(df)
